In [1]:
import requests,json,os,time
import random,pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx
import geojson
apiKey=os.getenv('CKANAPIKEY')
if apiKey is None:
    raise EnvironmentError("Failed because {} is not set.".format('CKANAPIKEY'))

In [2]:
# print(pd.__version__)

In [3]:
mainDir = '/Users/iperezx/Documents/sage-commons/sage-commons/'
#sageCommonsURL='http://127.0.0.1:5000'
sageCommonsURL='http://hotshot.sdsc.edu:5000'
headers = {'Authorization': apiKey}
orgName= 'sage'

### get node and plugin data from endpoint

In [4]:
nodeURL = 'https://sage-data-api.nautilus.optiputer.net/api/v1/nodes-data'

response = requests.request("GET", nodeURL)
nodeJSON = response.json()
# print(nodeJSON['data'])

nodeDataDF = pd.DataFrame.from_dict(nodeJSON['data'], orient='columns')
nodeDataDF['lon'] = pd.to_numeric(nodeDataDF['lon'],errors='coerce')
nodeDataDF['lat'] = pd.to_numeric(nodeDataDF['lat'],errors='coerce')
nodeDataDF.rename(columns={'id':'nodeID'},inplace=True)

geometry = [Point(xy) for xy in zip(nodeDataDF.lon, nodeDataDF.lat)]
nodeDataGDF = gpd.GeoDataFrame(nodeDataDF.drop(['lon', 'lat'], axis=1), crs="EPSG:4326", geometry=geometry)
nodeDataGDF.rename(columns={'id':'nodeID'},inplace=True)

In [5]:
nodeDataGDF

,name,nodeID,status,provisionDate,OSVersion,serviceTag,SpecialDevices,BiosVersion,geometry
0,Sage-NEON-01,4cd98fc4d2a8,Up,10/22/20,dell-1.0.0.local-6da62a8,79BBZ23,N/A,2.8.2,POINT (-105.24585 40.01631)
1,Sage-NEON-02,4cd98fc67b75,Up,10/22/20,dell-1.0.0.local-6da62a8,79B9Z23,Camera (10.102.144.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631)
2,Sage-NEON-03,4cd98fc68d9c,Up,10/30/20,dell-1.0.0.local-6da62a8,79BDZ23,Camera (10.110.48.41),2.8.2,POINT (-105.24585 40.01631)
3,Sage-NEON-04,4cd98fc67734,Up,11/2/20,dell-1.0.0.local-6da62a8,79BCZ23,Camera (10.114.16.41),2.8.2,POINT (-105.24585 40.01631)
4,Sage-NEON-05,4cd98fc67a3d,Up,11/3/20,dell-1.0.0.local-6da62a8,79BGZ23,Camera (10.116.128.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631)
5,Sage-NEON-06,4cd98fc67e76,Down/Transport,11/3/20,dell-1.0.0.local-6da62a8,79B8Z23,N/A,2.8.2,POINT (-105.24585 40.01631)
6,Sage-NEON-07,4cd98fc68549,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631)
7,Sage-NEON-08,4cd98fc686c9,Down/Transport,11/4/20,dell-1.0.0.local-6da62a8,79BFZ23,N/A,2.8.2,POINT (-105.24585 40.01631)
8,Sage-Dev-01,4cd98fb00025,Up,11/16/20,dell-1.0.0.local-6da62a8,8M0BN23,N/A,2.8.2,POINT (-87.97216 41.72586)
9,Sage-Dev-02,4cd98fa8be25,Up,10/22/20,dell-1.0.0.local-6da62a8,8M09N23,N/A,2.8.2,POINT (-87.97216 41.72586)


In [6]:
# ax = nodeDataGDF.plot(figsize=(20, 10))
# ctx.add_basemap(ax,crs=nodeDataGDF.crs.to_string())

In [7]:
pluginURL = 'https://sage-data-api.nautilus.optiputer.net/api/v1/plugin-data'

response = requests.request("GET", pluginURL)
pluginJSON = response.json()

pluginDataDF = pd.DataFrame.from_dict(pluginJSON['data'], orient='columns')
pluginDataDF['timestamp'] = pd.to_datetime(pluginDataDF['timestamp'])
pluginDataDF = pluginDataDF.drop('metadata',1).join(pd.DataFrame.from_dict(pluginDataDF.metadata.to_dict(),orient='index'))
pluginDataDF.rename(columns={'node':'nodeID'},inplace=True)
pluginDataDF

,timestamp,name,value,nodeID,plugin,sensorID
0,2021-04-01 07:00:00+00:00,env.temperature.gen,0.753540,4cd98fadebf3,metsense:1.0.2,000000000
1,2021-04-01 08:00:00+00:00,env.temperature.gen,0.762183,4cd98fc67a3d,metsense:1.0.2,000000000
2,2021-04-01 09:00:00+00:00,env.temperature.gen,0.270263,4cd98fc68549,metsense:1.0.2,000000000
3,2021-04-01 10:00:00+00:00,env.temperature.gen,0.347285,4cd98fadebf3,metsense:1.0.2,000000000
4,2021-04-01 11:00:00+00:00,env.temperature.gen,0.164852,4cd98fc68549,metsense:1.0.2,000000000
...,...,...,...,...,...,...
308,2021-04-14 03:00:00+00:00,env.temperature.gen,0.935313,4cd98fadebf3,metsense:1.0.2,000000000
309,2021-04-14 04:00:00+00:00,env.temperature.gen,0.021956,4cd98fc68549,metsense:1.0.2,000000000
310,2021-04-14 05:00:00+00:00,env.temperature.gen,0.039710,4cd98fc67a3d,metsense:1.0.2,000000000
311,2021-04-14 06:00:00+00:00,env.temperature.gen,0.507513,4cd98fc68549,metsense:1.0.2,000000000


In [8]:
# merge node data and plugin data
#pluginDataDF
combinedData = pluginDataDF.merge(nodeDataGDF, how='inner', on='nodeID')
combinedData

,timestamp,name_x,value,nodeID,plugin,sensorID,name_y,status,provisionDate,OSVersion,serviceTag,SpecialDevices,BiosVersion,geometry
0,2021-04-01 07:00:00+00:00,env.temperature.gen,0.753540,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586)
1,2021-04-01 10:00:00+00:00,env.temperature.gen,0.347285,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586)
2,2021-04-01 17:00:00+00:00,env.temperature.gen,0.926269,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586)
3,2021-04-01 22:00:00+00:00,env.temperature.gen,0.887408,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586)
4,2021-04-02 02:00:00+00:00,env.temperature.gen,0.596819,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2021-04-13 16:00:00+00:00,env.temperature.gen,0.191574,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631)
309,2021-04-13 21:00:00+00:00,env.temperature.gen,0.597471,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631)
310,2021-04-13 23:00:00+00:00,env.temperature.gen,0.106215,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631)
311,2021-04-14 04:00:00+00:00,env.temperature.gen,0.021956,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631)


In [9]:
# get sensor hardware
sensorHardwareURL = 'https://sage-data-api.nautilus.optiputer.net/api/v1/sensor-hardware-data'

response = requests.request("GET", sensorHardwareURL)
sensorHardwareJSON = response.json()
# print(sensorHardwareJSON['data'])

sensorHardwareDF = pd.DataFrame.from_dict(sensorHardwareJSON['data'], orient='columns')
sensorHardwareDF.rename(columns={'id':'sensorID'},inplace=True)
sensorHardwareDF

,sensorID,product_name,manufacture,sensor_types,link
0,000000000,bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
1,000000001,rg-15,Hydreon Corporation,rainfall measuring device,https://rainsensors.com/products/rg-15/
2,000000002,bme280,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
3,000000003,xnv-8081z,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnv-8081z.html
4,000000004,xnf-8010rv,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnf-8010rv.html
5,000000005,xnv-8080r,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnv-8080r.html
6,000000006,xnv-8082r,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnv-8082r.html


In [10]:
#merge sensor hardware data with pluginData
combinedData = combinedData.merge(sensorHardwareDF, how='inner', on='sensorID')
combinedData

,timestamp,name_x,value,nodeID,plugin,sensorID,name_y,status,provisionDate,OSVersion,serviceTag,SpecialDevices,BiosVersion,geometry,product_name,manufacture,sensor_types,link
0,2021-04-01 07:00:00+00:00,env.temperature.gen,0.753540,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
1,2021-04-01 10:00:00+00:00,env.temperature.gen,0.347285,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
2,2021-04-01 17:00:00+00:00,env.temperature.gen,0.926269,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
3,2021-04-01 22:00:00+00:00,env.temperature.gen,0.887408,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
4,2021-04-02 02:00:00+00:00,env.temperature.gen,0.596819,4cd98fadebf3,metsense:1.0.2,000000000,Sage-Dev-03,Up,10/22/20,dell-1.0.0.local-6da62a8,8M0CN23,N/A,2.8.2,POINT (-87.97216 41.72586),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2021-04-13 16:00:00+00:00,env.temperature.gen,0.191574,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
309,2021-04-13 21:00:00+00:00,env.temperature.gen,0.597471,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
310,2021-04-13 23:00:00+00:00,env.temperature.gen,0.106215,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
311,2021-04-14 04:00:00+00:00,env.temperature.gen,0.021956,4cd98fc68549,metsense:1.0.2,000000000,Sage-NEON-07,Up,11/4/20,dell-1.0.0.local-6da62a8,79B7Z23,Camera (10.117.48.41 & 10.110.48.41),2.8.2,POINT (-105.24585 40.01631),bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...


### create data resource

In [11]:
# get metadata from this plugin
pluginNameAll = combinedData.plugin.unique()[0].split(':')# will only return one
sensors = combinedData.sensor_types.unique()[0].split(",")
pluginName = pluginNameAll[0]
pluginVersion = pluginNameAll[1]
print('pluginName: ' + pluginName)
print('pluginVersion: ' + pluginVersion)
print( 'sensor: ' + str(sensors))

pluginName: metsense
pluginVersion: 1.0.2
sensor: ['relative humidity', 'barometric pressure', 'ambient temperature', 'gas']


In [12]:
# create geoJson from geometry unique Values
combinedData.geometry.unique()
multiPointVals = [(p.x,p.y) for p in combinedData.geometry.unique()]
print(multiPointVals)
geoJsonMultiPoints = geojson.MultiPoint(multiPointVals)
print(geoJsonMultiPoints)

[(-87.97216, 41.725861), (-105.24585, 40.01631)]
{"coordinates": [[-87.97216, 41.725861], [-105.24585, 40.01631]], "type": "MultiPoint"}


In [13]:
apiAction='/api/3/action/package_create'
url = sageCommonsURL+apiAction
title = pluginName + str(random.randint(1, 1e4))

name = title
spatialVal = geoJsonMultiPoints

extras = {'key': "spatial",'value': json.dumps(spatialVal) }
tags = [{'name':'plugin'}]

for x in sensors:
    tags.append({'name': x})

payload = {'owner_org': orgName,
           'title': title,
           'name' : name,
           'notes': 'Description of dataset',
           'tags' : tags,
           'extras': [extras]
           
          }

In [14]:
tags

[{'name': 'plugin'},
 {'name': 'relative humidity'},
 {'name': 'barometric pressure'},
 {'name': 'ambient temperature'},
 {'name': 'gas'}]

In [15]:
response = requests.request("POST", url, headers=headers, json = payload)
jsonResponseDS = response.json()
print(response)
print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

<Response [200]>
{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=package_create",
    "result": {
        "author": null,
        "author_email": null,
        "creator_user_id": "abbb7828-b01e-41e4-b9fb-00413a8336ca",
        "extras": [
            {
                "key": "spatial",
                "value": "{\"type\": \"MultiPoint\", \"coordinates\": [[-87.97216, 41.725861], [-105.24585, 40.01631]]}"
            }
        ],
        "groups": [],
        "id": "3d58745d-c9d0-4c19-a125-54a286937bc6",
        "isopen": false,
        "license_id": null,
        "license_title": null,
        "maintainer": null,
        "maintainer_email": null,
        "metadata_created": "2021-06-15T23:33:57.195567",
        "metadata_modified": "2021-06-15T23:33:57.195578",
        "name": "metsense5560",
        "notes": "Description of dataset",
        "num_resources": 0,
        "num_tags": 5,
        "organization": {
            "approval_status": "approved",
         

### create data source

In [16]:
apiAction='/api/3/action/sagecommons_create'
packageID = jsonResponseDS['result']['id']
url = sageCommonsURL+apiAction

resourceName = pluginName
resourceDescription = pluginName + ' inference data'
dataURL = pluginURL
# query = {"start": "-10m",
#         "filter": {
#             "name": "test"
#         }
# }

resource = {'package_id': packageID,
            'name' : resourceName,
            'description': resourceDescription,
            'url': dataURL,
            'format':'JSON',
            #'query':query,
           }


payload = {'resource': resource,
          }

# start = time.time() # start timing
response = requests.request("POST", url, headers=headers, json = payload)
# end = time.time()
# elapseTime = abs(end-start)
# print('Elapsed time: ' + str(elapseTime))

In [17]:
jsonResponse = response.json()
print(response)
print(response.text)
print(json.dumps(jsonResponse, indent=4, sort_keys=True))

<Response [200]>
{"help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=sagecommons_create", "success": true, "result": {"resource": {"mimetype": null, "name": "metsense", "format": "JSON", "url": "https://sage-data-api.nautilus.optiputer.net/api/v1/plugin-data", "package_id": "3d58745d-c9d0-4c19-a125-54a286937bc6", "description": "metsense inference data"}, "resource_id": "4e622b68-50d3-41f6-8695-81c1c75c1f38"}}
{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=sagecommons_create",
    "result": {
        "resource": {
            "description": "metsense inference data",
            "format": "JSON",
            "mimetype": null,
            "name": "metsense",
            "package_id": "3d58745d-c9d0-4c19-a125-54a286937bc6",
            "url": "https://sage-data-api.nautilus.optiputer.net/api/v1/plugin-data"
        },
        "resource_id": "4e622b68-50d3-41f6-8695-81c1c75c1f38"
    },
    "success": true
}


In [18]:
apiAction='/api/3/action/resource_view_list'
resourceID = jsonResponse['result']['resource_id']
url = sageCommonsURL+apiAction


payload = {'id': resourceID,
          }

response = requests.request("GET", url, headers=headers, json = payload)
jsonResponse = response.json()
print(json.dumps(jsonResponse, indent=4, sort_keys=True))

{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=resource_view_list",
    "result": [
        {
            "description": "",
            "id": "a8b48d8c-f6b0-449d-b896-2a27355251a4",
            "package_id": "3d58745d-c9d0-4c19-a125-54a286937bc6",
            "resource_id": "4e622b68-50d3-41f6-8695-81c1c75c1f38",
            "title": "Text",
            "view_type": "text_view"
        },
        {
            "description": null,
            "id": "7431b4d0-67eb-4345-9039-5210af7d0b38",
            "package_id": "3d58745d-c9d0-4c19-a125-54a286937bc6",
            "resource_id": "4e622b68-50d3-41f6-8695-81c1c75c1f38",
            "title": "Sage Interface View",
            "view_type": "sageinterfaceview"
        }
    ],
    "success": true
}


In [19]:
jsonResponse['result'][0]['id']

'a8b48d8c-f6b0-449d-b896-2a27355251a4'

In [20]:
apiAction='/api/3/action/resource_view_delete'
viewID = jsonResponse['result'][0]['id']
url = sageCommonsURL+apiAction


payload = {'id': viewID,
          }

response = requests.request("POST", url, headers=headers, json = payload)
jsonResponse = response.json()
print(json.dumps(jsonResponse, indent=4, sort_keys=True))

{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=resource_view_delete",
    "result": null,
    "success": true
}
